In [ ]:

!pip install openai PyPDF2 pandas tqdm

import os, pandas as pd
from tqdm import tqdm
from PyPDF2 import PdfReader
from openai import OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()     # upload "state_of_ai.pdf"
pdf_path = list(uploaded.keys())[0]

reader = PdfReader(pdf_path)
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"

print("✅ Loaded", len(text.split()), "words from report")

Saving Q3FY25-CFO-Commentary.pdf to Q3FY25-CFO-Commentary.pdf
✅ Loaded 2571 words from report


In [ ]:
context = " ".join(text.split()[:2571])
print(context[:800])   # preview


CFO Commentary on Third Quarter Fiscal 2025 Results Q3 Fiscal 2025 Summary GAAP ($ in millions, except earnings per share)Q3 FY25 Q2 FY25 Q3 FY24 Q/Q Y/Y Revenue $35,082 $30,040 $18,120 Up 17% Up 94% Gross margin 74.6 % 75.1 % 74.0 % Down 0.5 pts Up 0.6 pts Operating expenses $4,287 $3,932 $2,983 Up 9% Up 44% Operating income $21,869 $18,642 $10,417 Up 17% Up 110% Net income $19,309 $16,599 $9,243 Up 16% Up 109% Diluted earnings per share $0.78 $0.67 $0.37 Up 16% Up 111% Non-GAAP ($ in millions, except earnings per share)Q3 FY25 Q2 FY25 Q3 FY24 Q/Q Y/Y Revenue $35,082 $30,040 $18,120 Up 17% Up 94% Gross margin 75.0 % 75.7 % 75.0 % Down 0.7 pts -- Operating expenses $3,046 $2,792 $2,026 Up 9% Up 50% Operating income $23,276 $19,937 $11,557 Up 17% Up 101% Net income $20,010 $16,952 $10,020 U


In [ ]:
questions = [
    "What was NVIDIA's total revenue in Q3 FY25?",
    "Which segment contributed the most to revenue growth?",
    "How did the Data Center business perform compared to last year?",
    "What guidance did NVIDIA provide for Q4 FY25?",
    "What were the key drivers of NVIDIA's overall growth this quarter?"
]

In [21]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)



In [22]:
def make_prompts(question, context):
    return [
        f"Answer this question based only on the text below.\n\nText:\n{context}\n\nQuestion: {question}",
        f"You are a financial analyst. Using the report below, give a concise, data-backed answer.\n\n{context}\n\nQuestion: {question}",
        f"Summarize the answer in three bullet points, using only figures from the report.\n\n{context}\n\nQuestion: {question}",
        f"Quote relevant phrases from the report to justify your answer.\n\n{context}\n\nQuestion: {question}",
        f"If the question cannot be answered from the report, reply 'Not found in document.' Otherwise, give one-sentence answer.\n\n{context}\n\nQuestion: {question}"
    ]

In [23]:
results = []
for q in tqdm(questions):
    for i, prompt in enumerate(make_prompts(q, text), 1):  # Using full text
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        answer = response.choices[0].message.content.strip()
        results.append({
            "Question": q,
            "Prompt_ID": i,
            "Response": answer
        })

df = pd.DataFrame(results)
print("✅ Completed all prompts. Showing sample:")
display(df.head())

100%|██████████| 5/5 [00:28<00:00,  5.62s/it]

✅ Completed all prompts. Showing sample:


,Question,Prompt_ID,Response
0,What was NVIDIA's total revenue in Q3 FY25?,1,"NVIDIA's total revenue in Q3 FY25 was $35,082 million."
1,What was NVIDIA's total revenue in Q3 FY25?,2,NVIDIA's total revenue in Q3 FY25 was $35.082 billion.
2,What was NVIDIA's total revenue in Q3 FY25?,3,- Total revenue in Q3 FY25 was $35.1 billion\n- Data Center revenue in Q3 FY25 was $30.8 billion\n- Compute revenue in Q3 FY25 was $27.6 billion
3,What was NVIDIA's total revenue in Q3 FY25?,4,"NVIDIA's total revenue in Q3 FY25 was $35.1 billion. \n\nThis is supported by the statement: ""Revenue was a record $35.1 billion, up 94% from a year ago and up 17% sequentially."""
4,What was NVIDIA's total revenue in Q3 FY25?,5,NVIDIA's total revenue in Q3 FY25 was $35.1 billion.


In [ ]:
df.to_csv("nvidia_cfo_prompt_experiments.csv", index=False)
from google.colab import files
files.download("nvidia_cfo_prompt_experiments.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>